# 0803-博客项目-分页模块

## 一、首先搭建博客基本环境

### 创建项目

```python
django-admin startproject blog
```

### 创建应用

```python
python manage.py startapp booktest
```

### 配置settings.py

- **搭建应用**
```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'booktest',   # 修改点
]
```

- **配置templates模板查找路径**
```python
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [os.path.join(BASE_DIR, 'templates')],   # 修改点
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
```

- **连接MySQL数据库**
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',   # 链接MySQL
        'NAME': 'blog_test',             # 数据库名
        'USER': 'root',                 # 用户名
        'PASSWORD': '123456',             # 密码
        'HOST': '127.0.0.1',             # 数据库IP
        'POST': '3306',                # 数据库端口号
    }
}
```

- **修改中文和时区配置**  

```python
LANGUAGE_CODE = 'zh-hans'  # 中文字符集设置

TIME_ZONE = 'Asia/Shanghai'  # 时区设置

USE_I18N = True

USE_L10N = True

USE_TZ = False    # 修改
```

- **配置静态文件查找路径**
```python
STATICFILES_DIRS = [
    os.path.join(BASE_DIR, 'static')
]
```

- **配置多媒体文件查找路径**
```python
MEDIA_ROOT = os.path.join(BASE_DIR, 'static')
```

### 配置主路由

```python
from django.conf.urls import url, include
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'', include('booktest.urls'))
]
```

### 创建模板、静态、多媒体文件

三个目录均是根据`settings.py`中设置查找路径进行创建的

在项目目录下创建模板存放目录`templates`,用于存放HTML模板

在项目目录下创建静态文件存放目录`static`,用于存放js、css、img等渲染文件

因`settings.py`中多媒体查找路径直接设置在`static`目录中，故无需再创建多媒体目录


-----------------------------

最后并将HTML、static全部拖拽到项目中相应位置，导入前端显示页面


### 配置子路由

在`booktest`应用目录下创建`urls.py`子路由  

```python
from django.conf.urls import url
from .views import *

urlpatterns = [
    url(r'^$', index),
    # 路由连接主页
    url(r'^category/(\d+)/$', category),
    # 路由连接每一个分类页
]
```

### 配置模型类models

实现博客项目基本功能，我们暂时需要定义分类表，用户表和文章表  

```python
from django.db import models
from django.contrib.auth.models import User
# Create your models here.

# 分类表
class Category(models.Model):
    name = models.CharField(max_length=30, verbose_name="类名")

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = "分类表"
        verbose_name_plural = verbose_name


# 定义用户表
class UserProfile(models.Model):
    # 用户名与django中user表一对一关联
    user = models.OneToOneField(User, verbose_name="用户名")
    nick_name = models.CharField(max_length=30, verbose_name="昵称")
    qq = models.CharField(max_length=10, verbose_name="QQ号")
    head_img = models.ImageField(upload_to='head_img', verbose_name="头像")

    def __str__(self):
        return self.nick_name

    class Meta:
        verbose_name = "用户表"
        verbose_name_plural = verbose_name


class Article(models.Model):
    title = models.CharField(max_length=40, verbose_name="标题")
    # 分类外键关联分类表
    category = models.ForeignKey(Category, verbose_name="分类")
    jianjie = models.CharField(max_length=30, verbose_name="简介")
    article_pic = models.ImageField(upload_to='article_img', verbose_name="文章图片")
    publish_time = models.DateTimeField(auto_now_add=True, verbose_name="发布时间")
    # 参数auth_now_add获取当前时间
    
    # 作者外键关联用户表
    author = models.ForeignKey(UserProfile, verbose_name="作者")
    liulanshu = models.IntegerField(default=0, verbose_name="点击量")
    # default默认值
    neirong = models.TextField(max_length=1000, verbose_name="文章内容")
    # TextField大文本类型
    tuijian = models.BooleanField(default=True, verbose_name="是否推荐")
    # 布尔型，默认推荐

    def __str__(self):
        return self.title

    class Meta:
        verbose_name = "文章表"
        verbose_name_plural = verbose_name

```

### 注册后台管理

将models模型类注册到后台管理中，以便添加修改数据

```python
from django.contrib import admin
from .models import *
# Register your models here.


admin.site.register(Category)    # 注册分类表
admin.site.register(UserProfile)  # 注册用户表
admin.site.register(Article)     # 注册文章表
```

### 配置视图函数views

先将HTML渲染，检查应用是否成功

```python
from django.shortcuts import render
from .models import *
# Create your views here.


def index(request):
    return render(request, 'booktest/index.html')


def category(request, id):
    pass
```

### 生成&执行迁移文件

- 生成迁移文件
```python
python manage.py makemigrations
```

- 执行迁移文件
```python
python manage.py migrate
```


### 创建超级管理员用户
```python 
python manage.py createsuperuser
```


### 启动服务
```python
python manage.py runserver
```

### 后台添加数据

进入后台管理系统，输入账号密码，在分类表、用户表和文章表中添加一部分数据，以便后期进行处理，展示

       
                         -----------------------------------------------------------------

## 二、按分类展示文章

> 实现目的  
> 在前端页面中显示所有的分类信息，点击各自的分类，跳转页面，展示出该分类的所有文章


### 修改视图函数views
```python
from django.shortcuts import render
from .models import *

def index(request):
    # 获取分类表中所有分类信息
    all_category = Category.objects.all()
    # 获取所有文章信息
    all_article = Article.objects.all()
    return render(request, 'booktest/index.html', locals())


def category(request, id):
    # 获取分类表中所有分类信息
    all_category = Category.objects.all()
    # 获取该分类下的所有文章
    all_article = Article.objects.filter(category_id=int(id))
    # 跳转到category.html页面
    return render(request, 'booktest/category.html', locals())
```


### 创建category.html

category继承index页面

```
{% extends 'booktest/index.html' %}
```

### 修改index.html

1. 在HTML中添加模板for语言，循环遍历出所有标签的分类，  
同时实现点击分类名，跳转到相应的分类页，以显示该分类的所有文章。

```html
{% for value in all_category %}
    <a href="/category/{{ value.id }}">{{ value.name }}</a>
{% endfor%}
```

2. 在HTML中添加模板for语言，循环遍历出所有文章，并显示到页面

```html
<h2>最新文章</h2>
{% for article in all_article %}
 <div class="blogs">
   <ul>
     <h3><a href="">{{ article.title }}</a></h3>
     <p>{{ article.jianjie }}</p>
     <p><img src="/static/{{ article.article_pic }}" style="height: 50px"></p>
     <p class="autor">
         <span class="lm f_l"><a href="/">{{ article.category }}</a></span>
         <span class="dtime f_l">{{ article.publish_time }}</span>
         <span class="viewnum f_r">浏览（<a href="/">{{ article.liulanshu }}</a>）</span>
         <span class="pingl f_r">评论(<a href="/"></a>)</span>
     </p>
   </ul>
 </div>
{% endfor %}
```

> 这里的HTML设置仅供查看模板语言配置，前端效果以模板实际显示为主

### 查看效果

启动服务，查看效果是否实现  

>注意超链接、图片等的路径问题  
>注意当多媒体目录更改，要修改后续所有关于多媒体路径有关的配置

                  
                                --------------------------------------------------------------

## 三、文章分页效果

> 实现目的  
> 文章信息不会全部罗列到一个页面中，  
> 而是将所有的文章按每页固定显示的条目数显示文章，  
> 并实现下一页、上一页、总页数、当前所在页数等功能  

### 分页概念

Django提供了数据分页的类，这些类被定义在`django/core/paginator.py`文件中  
分页的类实际中会使用两个实例化对象：  
- Paginator对象  
    用于对列进行一页n条数据的分页运算


- Page对象  
    用于表示第m页的数据信息
    
**Paginator对象中的方法和属性**：

1. init(list，int)方法：返回分页对象，参数为列表数据，每页显示的数据条数
```python
paginator = Paginator([1,2,3,4,5,6,7,8,9,10], 2)
# 实例化paginator对象，参数列表为所有的数据，整数为每页显示的数据条数
```
2. count属性：返回对象总数
```python
paginator.count
```

3. num_pages属性：返回页面总数
```python
paginator.num_pages
```

4. page_range属性：返回页码数列表，从1开始
```python 
paginator.page_range
# 返回range对象，以便调用
```

5. page(m)方法：返回Page对象，表示第m页的数据，下标以1开始
```python
page = paginator.page(2)
# 此方法实现实例化Page对象，并返回第2页的数据
```

**Page对象中的方法和属性：**

> Paginator对象调用`page()`方法可是实例化Page对象，以便调用Page的方法和属性

1. object_list属性：返回当前页对象的列表
```python
page.object_list
# 返回一个QuerySet对象，返回当前页数据的列表
```

2. number属性：返回当前是第几页，从1开始
```python 
page.number
# 返回一个整型，是当前处于的页数
```

3. has_next()方法：如果有下一页返回True，一般用于前端模板
```python
page.has_next()
# 布尔型，不过主要不这么用，一般吧page传递到前端，前端用if判断语句进行后续业务
```

4. has_previous()方法：如果有上一页返回True，一般用于前端模板
```python
page.has_previous()
```

5. len()方法：返回当前页面对象的个数
```python
page.__len__()
# 返回整型，当前页面数据对象的个数
```

6. paginator属性：当前页对应的Paginator对象
```python
page.paginator
# 此方法将Page对象转化为Paginator对象，以便使用该对象的方法
```

    

### 导入分页管理器

```python
from django.core.paginator import Paginator
```

### 修改视图函数views

```python
from django.shortcuts import render
from .models import *
# Create your views here.
# 导入分页管理器
from django.core.paginator import Paginator

def index(request):
    all_category = Category.objects.all()
    all_article = Article.objects.all()
    return render(request, 'booktest/index.html', locals())


def category(request, id):
    all_category = Category.objects.all()
    # 获取分类下的所有文章
    all_article1 = Article.objects.filter(category_id=int(id))
    # 实例化分页对象，Panginator对象，
    # 要展示的所有数据列表和每一页需要展示的数据条数作为参数传递进去
    paginator = Paginator(all_article1, 2)
    # 获取前端HTML传过来的页码，page无参数默认赋值1
    page = request.GET.get('page', 1)
    # 返回当前页的数据,Page对象
    all_article = paginator.page(page)
    # 返回页码列表
    page_list = paginator.page_range
    # 返回页码总数
    page_zong = paginator.num_pages
    return render(request, 'booktest/category.html', locals())

```

### 配置HTML模板

使用if判断用于实现上一页、下一页动态显示，其中显示页数列表
```html
{% if all_article.has_previous %}
    <li class="previous-off"><a href="?page={{ all_article.number|add:-1 }}">&laquo;上一页</a></li>
{% endif %}

{% for page in page_list %}
     <a href="?page={{ page }}">{{ page }}</a>
{% endfor %}

{% if all_article.has_next %}
     <li class="next"><a href="?page={{ all_article.number|add:1 }}">下一页 &raquo;</a></li>
{% endif %}
```

显示总页码数和当前页数设置
```html
<h1 style="font-size: 16px">总页码：共{{ page_zong }} 页</h1>
<h1 style="font-size: 16px">当前页：第{{ all_article.number }}页</h1>
```

> 这里的HTML设置仅供查看模板语言配置，前端效果以模板实际显示为主

### 启动服务

启动服务查看运行效果，分类展示文章信息，文章信息包括：标题、所属分类、作者、图片、简介、发布日期、浏览量等。以及分页展示是否成功